In [76]:
import numpy as np
import pandas as pd
import emoji
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
print("Done")

Done


In [77]:
emoji_dict = {'happy': "\u2764\uFE0F", 'baseball': ":baseball:", 'smile': ":beaming_face_with_smiling_eyes:",
              'sad': ":downcast_face_with_sweat:", 'food': ":fork_and_knife:"}
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix]))

happy ❤️
baseball ⚾
smile 😁
sad 😓
food 🍴


In [78]:
# Load data
train = pd.read_csv('training.csv', header=None)
test = pd.read_csv('test.csv', header=None)

# Extract features and labels
X_train = train.values[:, 0]
Y_train = train.values[:, 1]
X_test = test.values[:, 0]
Y_test = test.values[:, 1]

In [79]:
# Convert labels to numerical values
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_train_categorical = to_categorical(Y_train_encoded)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad sequences to make them of equal length
max_length = 10  # Choose an appropriate value based on your data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length)

In [80]:
# Load pre-trained embeddings
embeddings_index = {}
with open('./glove.6B.50d.txt', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train_padded.shape[0], max_length, 50))
embedding_matrix_test = np.zeros((X_test_padded.shape[0], max_length, 50))

for ix in range(X_train_padded.shape[0]):
    for ij in range(min(max_length, len(X_train_padded[ix]))):
        word_index = X_train_padded[ix][ij]
        if word_index != 0:  # Skip padding index
            word = tokenizer.index_word[word_index]
            embedding_matrix_train[ix][ij] = embeddings_index.get(word, np.zeros((50,)))

for ix in range(X_test_padded.shape[0]):
    for ij in range(min(max_length, len(X_test_padded[ix]))):
        word_index = X_test_padded[ix][ij]
        if word_index != 0:  # Skip padding index
            word = tokenizer.index_word[word_index]
            embedding_matrix_test[ix][ij] = embeddings_index.get(word, np.zeros((50,)))

In [81]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(max_length, 20), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_14 (LSTM)              (None, 10, 128)           91648     
                                                                 
 dropout_14 (Dropout)        (None, 10, 128)           0         
                                                                 
 lstm_15 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_15 (Dropout)        (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 5)                 645       
                                                                 
Total params: 223877 (874.52 KB)
Trainable params: 223877 (874.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [82]:
# Split the data into training and validation sets
X_train_final, X_val, Y_train_final, Y_val = train_test_split(embedding_matrix_train, Y_train_categorical, test_size=0.2, random_state=42)

# Training model
hist = model.fit(X_train_final, Y_train_final, epochs=50, batch_size=32, validation_data=(X_val, Y_val))
print("Ran full")

Epoch 1/50
5/5 [==============================] - 5s 200ms/step - loss: 1.5700 - accuracy: 0.3585 - val_loss: 1.5725 - val_accuracy: 0.2750
Epoch 2/50
5/5 [==============================] - 0s 20ms/step - loss: 1.5196 - accuracy: 0.3648 - val_loss: 1.5821 - val_accuracy: 0.3750
Epoch 3/50
5/5 [==============================] - 0s 20ms/step - loss: 1.4942 - accuracy: 0.3774 - val_loss: 1.5627 - val_accuracy: 0.2750
Epoch 4/50
5/5 [==============================] - 0s 19ms/step - loss: 1.4618 - accuracy: 0.3711 - val_loss: 1.5127 - val_accuracy: 0.3500
Epoch 5/50
5/5 [==============================] - 0s 19ms/step - loss: 1.4190 - accuracy: 0.3585 - val_loss: 1.4656 - val_accuracy: 0.3750
Epoch 6/50
5/5 [==============================] - 0s 20ms/step - loss: 1.3436 - accuracy: 0.4969 - val_loss: 1.4250 - val_accuracy: 0.3000
Epoch 7/50
5/5 [==============================] - 0s 19ms/step - loss: 1.2776 - accuracy: 0.4906 - val_loss: 1.3957 - val_accuracy: 0.3000
Epoch 8/50
5/5 [==========

In [83]:
"""# Prediction of trained model
pred = model.predict(embedding_matrix_test)
# print(emoji.emojize(emoji_dict[pred[0]]))
print("Done predicting")

# Convert labels to numerical values for test data
Y_test_encoded = label_encoder.transform(Y_test)
Y_test_categorical = to_categorical(Y_test_encoded)
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(embedding_matrix_test, Y_test_categorical)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')"""

# Assuming pred is the predicted labels from your model.predict
pred = model.predict(embedding_matrix_test)

# Flatten the pred array
pred_flat = pred.flatten()

# Convert labels to numerical values for test data
Y_test_encoded = label_encoder.transform(Y_test)
Y_test_categorical = to_categorical(Y_test_encoded)
# Evaluate on test data
test_loss, test_accuracy = model.evaluate(embedding_matrix_test, Y_test_categorical)
print(f'Model Evaluation - Test Accuracy: {test_accuracy * 100:.2f}%')



2/2 [==============================] - 0s 8ms/step - loss: 1.2894 - accuracy: 0.7391
Model Evaluation - Test Accuracy: 73.91%


In [84]:
# Get predictions on the test set
Y_pred = model.predict(embedding_matrix_test)

# Convert Y_test to integers if it's not already
Y_test_int = label_encoder.transform(Y_test)

# Display some sample predictions
for i in range(20):
    print(f"Text: {X_test[i]}")
    
    true_label = label_encoder.inverse_transform([Y_test_int[i]])[0]
    predicted_label = label_encoder.inverse_transform(np.argmax([Y_pred[i]], axis=1))[0]
    
    print(f"True Label: {true_label} - Predicted Label: {predicted_label}\n")


2/2 [==============================] - 0s 5ms/step
Text: I want to eat
True Label: food - Predicted Label: food

Text: he did not answer
True Label: sad - Predicted Label: sad

Text: he got a raise
True Label: smile - Predicted Label: smile

Text: she got me a present
True Label: heart - Predicted Label: smile

Text: ha ha ha it was so funny
True Label: smile - Predicted Label: smile

Text: he is a good friend
True Label: heart - Predicted Label: smile

Text: I am upset
True Label: heart - Predicted Label: sad

Text: We had such a lovely dinner tonight
True Label: heart - Predicted Label: smile

Text: where is the food
True Label: food - Predicted Label: food

Text: Stop making this joke ha ha ha
True Label: smile - Predicted Label: smile

Text: where is the ball
True Label: baseball - Predicted Label: baseball

Text: work is hard
True Label: sad - Predicted Label: sad

Text: This girl is messing with me
True Label: sad - Predicted Label: sad

Text: are you serious ha ha
True Label: sm

In [75]:
#model.save('./model/best-model.pt')
#model.save('./model/best-model.h5')
model.save('mymodel.keras')
model.save()

print("Full run successful")

Full run successful
